# Ruta archivo

In [41]:
file_path = "C:/Users/Richard Salgado/Desktop/cv/farmers-protest-tweets-2021-2-4.json"

# Importación de librerías

In [42]:
import time
import pandas as pd
import numpy as np
import json  
import timeit
from memory_profiler import profile
from datetime import datetime, timedelta,date
import json
from typing import List, Tuple
from collections import defaultdict
from collections import Counter
from datetime import datetime
import emoji



# Importación de funciones

Se importan las funciones creadas en los archivos .py,
las cuales corresponden a las funciones optimizadas en tiempo y memoria de uso,
y también a la función que permite descargar la data de tweets utilizando solo la ruta del archivo.

In [43]:
from load_tweets import load_tweets

In [44]:

from q1_time import q1_time
from q1_memory import q1_memory
from q2_time import q2_time
from q2_memory import q2_memory
from q3_time import q3_time
from q3_memory import q3_memory

# Data input

In [45]:
tweets = load_tweets(file_path)

Resumen tweet[0]:

'url': La URL del tweet original en Twitter. Puedes acceder al tweet completo visitando este enlace.

'date': La fecha y la hora en que se publicó el tweet, en formato UTC (Tiempo Universal Coordinado). En este caso, el tweet fue publicado el 24 de febrero de 2021, a las 09:23:35 UTC.

'content': El contenido del tweet. En este caso, el autor (Arjun Singh Panam) está expresando su descontento con la policía y el gobierno de la India, alegando que están intentando llevar a India de vuelta a un pasado horroroso a través de su tiranía. También incluye menciones a personas específicas (@narendramodi y @DelhiPolice) y varios hashtags relacionados con protestas de agricultores en India.

'renderedContent': Similar al campo 'content', pero contiene la URL acortada de un tweet mencionado (posiblemente retweeteado) por Arjun Singh Panam.

'id': El identificador único del tweet en Twitter.

'user': Información sobre el usuario que publicó el tweet, incluyendo el nombre de usuario, nombre de pantalla, descripción del perfil, fecha de creación de la cuenta, y más.

'outlinks': Enlaces externos mencionados en el tweet. En este caso, hay un enlace a otro tweet: https://twitter.com/ravisinghka/status/1364150844757860352.

'tcooutlinks': Enlaces acortados de Twitter mencionados en el tweet. En este caso, hay un enlace acortado https://t.co/es3kn0IQAF, que parece ser un enlace a un medio (imagen, video, etc.) adjunto al tweet.

'replyCount', 'retweetCount', 'likeCount', 'quoteCount': Cantidad de respuestas, retweets, likes y citas que ha recibido el tweet.

'conversationId': Identificador de la conversación a la que pertenece el tweet. Todos los tweets en una conversación tendrían el mismo 'conversationId'.

'lang': El idioma del tweet. En este caso, es 'en' que corresponde a inglés.

'source', 'sourceUrl', 'sourceLabel': Información sobre la aplicación o dispositivo desde el cual se publicó el tweet. En este caso, fue publicado desde la aplicación "Twitter for iPhone".

'media': Información sobre medios adjuntos al tweet (imágenes, videos, etc.). En este caso, no hay medios adjuntos.

'retweetedTweet', 'quotedTweet': Información sobre tweets retuiteados o citados. En este caso, hay información sobre un tweet citado. Puedes acceder al tweet citado visitando este enlace.

'mentionedUsers': Usuarios de Twitter mencionados en el tweet, con detalles sobre cada uno. En este caso, se mencionan a @narendramodi y @DelhiPolice en el tweet original, y @police_haryana y @DelhiPolice en el tweet citado.

In [46]:
tweets[0]

{'url': 'https://twitter.com/ArjunSinghPanam/status/1364506249291784198',
 'date': '2021-02-24T09:23:35+00:00',
 'content': 'The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. \n\n@narendramodi @DelhiPolice Shame on you. \n\n#ModiDontSellFarmers \n#FarmersProtest \n#FreeNodeepKaur https://t.co/es3kn0IQAF',
 'renderedContent': 'The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. \n\n@narendramodi @DelhiPolice Shame on you. \n\n#ModiDontSellFarmers \n#FarmersProtest \n#FreeNodeepKaur twitter.com/ravisinghka/st…',
 'id': 1364506249291784198,
 'user': {'username': 'ArjunSinghPanam',
  'displayname': 'Arjun Singh Panam',
  'id': 45091142,
  'description': 'Global Citizen, Actor, Director: Sky is the roof above my head, the world is the road I travel, love is my food & mother earth is my bed. Roy in @CosmosMovie',
  'rawDescription':

# Problema 1: Top 10 fechas con más tweets y usuarios con más publicaciones

## Supuestos:
* Se asume que tweet es lo mismo que publicación. 
* Top 10 fechas con más tweets: Considerando que cada tweet tiene una fecha asociada, se calcula para cada fecha, el número de tweets asociados. Finalmente se extraen las 10 primeras fechas con más tweets. 
* Top 10 usuarios con más publicaciones: Considerando que cada publicación tiene un usuario asociado, se calcula para cada usuario, el número de publicaciones asociadas. Finalmente se extraen los 10 primeros usuarios con más publicaciones.
* Se calcula el top de fechas y top de usuarios de forma independiente.


## Baseline función

* Este ejercicio es análogo a realizar un "groupby" por fecha y un "groupby" por usuario, acompañado de la función agregada "count" para cada uno, y finalmente ordenar para cada uno en base al número de publicaciones.
* Un ejemplo es el siguiente:

In [47]:

# Para cada tweet obtenemos su fecha y usuario respectivo
tweets_fechas = [datetime.strptime(tweet["date"], "%Y-%m-%dT%H:%M:%S%z").date() for tweet in tweets]
tweets_users = [tweet['user']['username'] for tweet in tweets] 
id_tweets = [i for i in range(len(tweets))]

# Corresponden a la misma cantidad de tweets, 117407
print(len(tweets_fechas),len(tweets_users),len(id_tweets))

# Se guarda la info en un df para realizar las funciones de agrupación y obtener el top 10 por fecha y usuario.
df_tweets = pd.DataFrame()
df_tweets['id_tweet'] = id_tweets
df_tweets['fecha'] = tweets_fechas
df_tweets['username'] = tweets_users

df_tweets.head(10)



117407 117407 117407


,id_tweet,fecha,username
0,0,2021-02-24,ArjunSinghPanam
1,1,2021-02-24,PrdeepNain
2,2,2021-02-24,parmarmaninder
3,3,2021-02-24,anmoldhaliwal
4,4,2021-02-24,KotiaPreet
5,5,2021-02-24,babli_708
6,6,2021-02-24,Varinde17354019
7,7,2021-02-24,BitnamSingh
8,8,2021-02-24,anmoldhaliwal
9,9,2021-02-24,SatThiara


In [48]:
# Agrupar por fecha y obtener top 10
top_10_fecha = df_tweets.groupby(['fecha']).agg({'id_tweet':'count'}).reset_index().sort_values(by='id_tweet',ascending=False).head(10)
top_10_fecha.columns = ['fecha','n° publicaciones']

# Agrupar por usuario y obtener top 10
top_10_user = df_tweets.groupby(['username']).agg({'id_tweet':'count'}).reset_index().sort_values(by='id_tweet',ascending=False).head(10)
top_10_user.columns = ['username','n° publicaciones']

In [49]:
top_10_fecha

,fecha,n° publicaciones
0,2021-02-12,12347
1,2021-02-13,11296
5,2021-02-17,11087
4,2021-02-16,10443
2,2021-02-14,10249
6,2021-02-18,9625
3,2021-02-15,9197
8,2021-02-20,8502
11,2021-02-23,8417
7,2021-02-19,8204


In [50]:
top_10_user

,username,n° publicaciones
20679,jot__b,1019
23788,rebelpacifist,850
8589,MaanDee08215437,830
5010,Gurpreetd86,636
4852,GurmVicky,597
24711,shells_n_petals,576
23238,preetysaini321,573
20093,ish_kayy,515
7619,KaurDosanjh1979,512
3580,DigitalKisanBot,490


In [51]:
top_10_total = pd.DataFrame()
top_10_total['fecha'] = top_10_fecha['fecha']
top_10_total['usuario'] = top_10_user['username'].to_list()
top_10_total

,fecha,usuario
0,2021-02-12,jot__b
1,2021-02-13,rebelpacifist
5,2021-02-17,MaanDee08215437
4,2021-02-16,Gurpreetd86
2,2021-02-14,GurmVicky
6,2021-02-18,shells_n_petals
3,2021-02-15,preetysaini321
8,2021-02-20,ish_kayy
11,2021-02-23,KaurDosanjh1979
7,2021-02-19,DigitalKisanBot


## Optimización de funciones

En base a las variables de uso de tiempo y uso de memoria, se construyen las funciones para poder optimizar la función de baseline. La idea es mantener el mismo resultado, pero con la mejora en las variables mencionadas.

In [13]:
import pandas as pd
from datetime import datetime
import memory_profiler
%load_ext memory_profiler

# Medir tiempo y memoria para la función original
result_original_time = %timeit -o q1_time(file_path)
result_original_memory = %memit -o q1_time(file_path)

# Medir tiempo y memoria para la función optimizada
result_optimized_time = %timeit -o q1_memory(file_path)
result_optimized_memory = %memit -o q1_memory(file_path)

# Crear un DataFrame con los resultados
results_df_1 = pd.DataFrame({
    'Función': ['q1_time', 'q1_memory'],
    'Tiempo (media)': [result_original_time.average, result_optimized_time.average],
    'Memoria (pico)': [result_original_memory.mem_usage[0], result_optimized_memory.mem_usage[0]]
})

# Mostrar el DataFrame
results_df_1

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
4.98 s ± 97.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 2435.57 MiB, increment: 896.36 MiB
5.43 s ± 167 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 2426.71 MiB, increment: 881.04 MiB


,Función,Tiempo (media),Memoria (pico)
0,q1_time,4.975269,2435.570312
1,q1_memory,5.427386,2426.707031


Optimización de tiempo

In [26]:
q1_t = q1_time(file_path)
q1_t

[(datetime.date(2021, 2, 12), 'jot__b'),
 (datetime.date(2021, 2, 13), 'rebelpacifist'),
 (datetime.date(2021, 2, 17), 'MaanDee08215437'),
 (datetime.date(2021, 2, 16), 'Gurpreetd86'),
 (datetime.date(2021, 2, 14), 'GurmVicky'),
 (datetime.date(2021, 2, 18), 'shells_n_petals'),
 (datetime.date(2021, 2, 15), 'preetysaini321'),
 (datetime.date(2021, 2, 20), 'ish_kayy'),
 (datetime.date(2021, 2, 23), 'KaurDosanjh1979'),
 (datetime.date(2021, 2, 19), 'DigitalKisanBot')]

Optimización de memoria

In [27]:
q1_m = q1_memory(file_path)
q1_m

[(datetime.date(2021, 2, 12), 'jot__b'),
 (datetime.date(2021, 2, 13), 'rebelpacifist'),
 (datetime.date(2021, 2, 17), 'MaanDee08215437'),
 (datetime.date(2021, 2, 16), 'Gurpreetd86'),
 (datetime.date(2021, 2, 14), 'GurmVicky'),
 (datetime.date(2021, 2, 18), 'shells_n_petals'),
 (datetime.date(2021, 2, 15), 'preetysaini321'),
 (datetime.date(2021, 2, 20), 'ish_kayy'),
 (datetime.date(2021, 2, 23), 'KaurDosanjh1979'),
 (datetime.date(2021, 2, 19), 'DigitalKisanBot')]

# Problema 2: Top 10 emojis más usados

## Supuestos:
* Emoji más usado: Podrían considerarse 3 opciones para calcular esta métrica:
    * Emojis más usados totales.
    * Emojis más usados por tweet.--> En este challenge se utiliza esta lógica.
    * Emojis más usados por usuario.
    * Emojis más usados por tweet y usuario.

 Con el fin de evitar el sesgo de repetir emojis en tweets, se considera "emoji más usado" el que más se repita en tweets distintos, independiente del usuario que lo haya publicado. Por ejemplo, si tenemos lo siguiente:
    * Tweet : '😂 😂 ❤️jajajaj'. 
Entonces, ambos emojis cuentan como "1" en la sumatoria total ('😂':1, '❤️':1).
* Ahora bien, si es el mismo usuario el que repite los emojis en distintos tweets, por ejemplo:
    * Usuario: Pedro
        * Tweet 1 : '😂 😂 ❤️ ❤️ jajajaj'.  ('😂':1, '❤️':1)
        * Tweet 2: '😂 😂😂 ❤️ ❤️ ❤️ hola mundo'. ('😂':1, '❤️':1)
        * Tweet 3: '😂 😂😂😂 ❤️ ❤️ adioss'. ('😂':1, '❤️':1)
    * Conteo final : '😂':3, '❤️':3

El usuario no va a influir en el conteo. Se menciona porque si se quisiera medir por ejemplo, los top 10 emojis más usados en el mundo, sería interesante evaluar esa métrica en función del emoji que más se repite en tweets y usuarios distintos, no solo tweets distintos.


## Baseline función

Este ejercicio es análogo a obtener los emojis únicos por tweet, y luego hacer un groupby por emoji, aplicar la función agrupada de conteo, y finalmente ordenar de mayor a menor para tener el top 10.

In [38]:
emojis_unicos_listas = [list(set([fila['emoji'] for fila in emoji.emoji_list(tweet['content'])])) for tweet in tweets]
emojis_unicos = [emoji for lista in emojis_unicos_listas for emoji in lista]

emojis_df = pd.DataFrame()
emojis_df['emoji'] = emojis_unicos

top_10_emoji = emojis_df.groupby('emoji').size().reset_index()
top_10_emoji.columns  = ['emoji','n° emoji']
top_10_emoji = top_10_emoji.sort_values(by='n° emoji',ascending=False).head(10)


## Optimización de funciones

En base a las variables de uso de tiempo y uso de memoria, se construyen las funciones para poder optimizar la función de baseline. La idea es mantener el mismo resultado, pero con la mejora en las variables mencionadas.

In [9]:
import pandas as pd
from datetime import datetime
import memory_profiler
%load_ext memory_profiler

# Medir tiempo y memoria para la función original
result_original_time = %timeit -o q2_time(file_path)
result_original_memory = %memit -o q2_time(file_path)

# Medir tiempo y memoria para la función optimizada
result_optimized_time = %timeit -o q2_memory(file_path)
result_optimized_memory = %memit -o q2_memory(file_path)

# Crear un DataFrame con los resultados
results_df_2 = pd.DataFrame({
    'Función': ['q2_time', 'q2_memory'],
    'Tiempo (media)': [result_original_time.average, result_optimized_time.average],
    'Memoria (pico)': [result_original_memory.mem_usage[0], result_optimized_memory.mem_usage[0]]
})

# Mostrar el DataFrame
results_df_2

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
14.7 s ± 237 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 2433.35 MiB, increment: 1113.39 MiB
16.3 s ± 1.41 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 2442.15 MiB, increment: 916.94 MiB


,Función,Tiempo (media),Memoria (pico)
0,q2_time,14.667872,2433.347656
1,q2_memory,16.327182,2442.148438


Optimización de tiempo

In [9]:
q2_t = q2_time(file_path)
q2_t

[('🙏', 3203),
 ('😂', 1387),
 ('🚜', 1334),
 ('🌾', 1298),
 ('❤️', 1205),
 ('✊', 1110),
 ('🇮🇳', 938),
 ('🤣', 759),
 ('👍', 634),
 ('🙏🏻', 580)]

Optimización de memoria

In [10]:
q2_m = q2_memory(file_path)
q2_m

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

# Problema 3: Top 10 histórico de usuarios más influyentes en función del conteo de menciones

## Supuestos:
* Usuarios más influyentes: Se considera "usuario más influyente" al que más se repite (menciones) en distintos tweets. De la misma forma que en el problema 2, se podría considerar también como el que más se repite(menciones) en distintos tweets y usuarios. Para efectos de este challenge, se considerará como el que más menciones posee por en distintos tweets.


## Baseline función

Este ejercicio es análogo a obtener los usuarios mencionados en cada tweet, y luego agrupar por usuario para finalmente realizar la función de conteo que permite obtener el top 10 de usuarios más mencionados.

* menciones_por_tweet = 
    * [ ['Pedro','Juan','Diego'], --> Menciones tweet 1
    * ['Diego','Pablo','Javier'] ] --> Menciones tweet 2

* menciones_total = ['Pedro','Juan','Diego', 'Diego','Pablo','Javier']

* Agrupar por usuario mencionado:
    * Diego : 2
    * Pedro : 1
    * Juan  : 1
    * Pablo  : 1
    * Javier : 1



In [34]:

mentions = [tweet["mentionedUsers"] for tweet in tweets]
usernames = [username['username'] for mention in mentions if mention is not None for username in mention if username is not None]

usernames_mentioned_df = pd.DataFrame()
usernames_mentioned_df['usuarios mencionados'] = usernames
top_10_mentioned = usernames_mentioned_df.groupby('usuarios mencionados').size().reset_index()
top_10_mentioned.columns = ['usuario mencionados','n° menciones']
top_10_mentioned = top_10_mentioned.sort_values(by='n° menciones',ascending=False).head(10)
top_10_mentioned

,usuario mencionados,n° menciones
12835,narendramodi,2265
4537,Kisanektamorcha,1840
6754,RakeshTikaitBKU,1644
6053,PMOIndia,1427
6613,RahulGandhi,1146
2974,GretaThunberg,1048
6899,RaviSinghKA,1019
13689,rihanna,986
8759,UNHumanRights,962
12599,meenaharris,926


## Optimización de funciones

En base a las variables de uso de tiempo y uso de memoria, se construyen las funciones para poder optimizar la función de baseline. La idea es mantener el mismo resultado, pero con la mejora en las variables mencionadas.

In [10]:
import pandas as pd
from datetime import datetime
import memory_profiler
%load_ext memory_profiler

# Medir tiempo y memoria para la función original
result_original_time = %timeit -o q3_time(file_path)
result_original_memory = %memit -o q3_time(file_path)

# Medir tiempo y memoria para la función optimizada
result_optimized_time = %timeit -o q3_memory(file_path)
result_optimized_memory = %memit -o q3_memory(file_path)

# Crear un DataFrame con los resultados
results_df_3 = pd.DataFrame({
    'Función': ['q3_time', 'q3_memory'],
    'Tiempo (media)': [result_original_time.best, result_optimized_time.best],
    'Memoria (pico)': [result_original_memory.mem_usage[0], result_optimized_memory.mem_usage[0]]
})

# Mostrar el DataFrame
results_df_3

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
3.62 s ± 111 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 2428.41 MiB, increment: 872.73 MiB
3.64 s ± 122 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 2417.52 MiB, increment: 890.01 MiB


,Función,Tiempo (media),Memoria (pico)
0,q3_time,3.395143,2428.410156
1,q3_memory,3.456128,2417.519531


Optimización de tiempo

In [8]:
q3_t = q3_time(file_path)
q3_t

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

Optimización de memoria

In [7]:
q3_m = q3_memory(file_path)
q3_m

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

# Resultados y conclusiones

## Problema 1 resultados

In [28]:
top_10_total

,fecha,usuario
0,2021-02-12,jot__b
1,2021-02-13,rebelpacifist
5,2021-02-17,MaanDee08215437
4,2021-02-16,Gurpreetd86
2,2021-02-14,GurmVicky
6,2021-02-18,shells_n_petals
3,2021-02-15,preetysaini321
8,2021-02-20,ish_kayy
11,2021-02-23,KaurDosanjh1979
7,2021-02-19,DigitalKisanBot


In [29]:
results_df_1

,Función,Tiempo (media),Memoria (pico)
0,q1_time,4.975269,2435.570312
1,q1_memory,5.427386,2426.707031


## Problema 2 resultados

In [39]:
top_10_emoji

,emoji,n° emoji
648,🙏,3203
562,😂,1387
667,🚜,1334
189,🌾,1298
113,❤️,1205
80,✊,1110
140,🇮🇳,938
731,🤣,759
354,👍,634
649,🙏🏻,580


In [ ]:
results_df_2

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
14.7 s ± 237 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 2433.35 MiB, increment: 1113.39 MiB
16.3 s ± 1.41 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 2442.15 MiB, increment: 916.94 MiB


,Función,Tiempo (media),Memoria (pico)
0,q2_time,14.667872,2433.347656
1,q2_memory,16.327182,2442.148438


## Problema 3 resultados

In [36]:
top_10_mentioned

,usuario mencionados,n° menciones
12835,narendramodi,2265
4537,Kisanektamorcha,1840
6754,RakeshTikaitBKU,1644
6053,PMOIndia,1427
6613,RahulGandhi,1146
2974,GretaThunberg,1048
6899,RaviSinghKA,1019
13689,rihanna,986
8759,UNHumanRights,962
12599,meenaharris,926


In [32]:
results_df_3

,Función,Tiempo (media),Memoria (pico)
0,q3_time,3.395143,2428.410156
1,q3_memory,3.456128,2417.519531


## Conclusiones

* Al abordar problemas de programación, hemos explorado dos enfoques cruciales: la optimización del tiempo de ejecución y la optimización de la memoria. Mediante el uso de listas de comprensión, pudimos agilizar los procesos y mejorar significativamente el tiempo de ejecución. Además, al priorizar ciclos for uno a uno, logramos reducir el consumo de memoria, especialmente en los problemas 1 y 3.

* Es vital destacar que no existe una solución única para todos los casos, y la elección entre optimización de tiempo y optimización de memoria depende en gran medida de la naturaleza única de cada problema. En el caso particular del problema 2, aunque se priorizó la optimización de memoria, la función optimizada en tiempo demostró un rendimiento superior, enfatizando la importancia de evaluar cuidadosamente las necesidades específicas de cada escenario.

* En conclusión, la optimización efectiva requiere un equilibrio cuidadoso entre el tiempo de ejecución y el uso de memoria. La comprensión profunda de las características individuales de cada problema guía la elección de la estrategia óptima, lo que subraya la necesidad de un enfoque flexible y adaptativo en la resolución de problemas de programación.

* Finalmente, es esencial destacar que el benchmarking realizado entre los enfoques de optimización implementados en este challenge es limitado en su alcance. Para obtener resultados más sólidos y adaptados al problema planteado, se recomienda una exploración más profunda de los métodos de optimización de algoritmos.

# Versiones de librerias

In [38]:
import sys

# Obtener las bibliotecas utilizadas en el código actual
used_libraries = [module for module in sys.modules if module.startswith('')]

# Imprimir las bibliotecas y sus versiones
for library in used_libraries:
    try:
        version = sys.modules[library].__version__
    except AttributeError:
        version = "No version information available"
    print(f"{library}=={version}")

sys==No version information available
builtins==No version information available
_frozen_importlib==No version information available
_imp==No version information available
_thread==No version information available
_warnings==No version information available
_weakref==No version information available
_io==No version information available
marshal==No version information available
nt==No version information available
winreg==No version information available
_frozen_importlib_external==No version information available
time==No version information available
zipimport==No version information available
_codecs==No version information available
codecs==No version information available
encodings.aliases==No version information available
encodings==No version information available
encodings.utf_8==No version information available
_signal==No version information available
encodings.latin_1==No version information available
_abc==No version information available
abc==No version information availab

In [28]:
import pkg_resources

installed_packages = pkg_resources.working_set
for package in installed_packages:
    print(f"{package.key}=={package.version}")

diff-match-patch==20200713
regex==2022.3.15
dask==2022.2.1
distributed==2022.2.1
fsspec==2022.2.0
tzdata==2022.2
scikit-learn-intelex==2021.20220215.102710
certifi==2021.10.8
imagecodecs==2021.8.26
tifffile==2021.7.2
daal4py==2021.5.0
pytz==2021.3
pywin32==302
setuptools==61.2.0
keyring==23.4.0
pyzmq==22.3.0
twisted==22.2.0
attrs==21.4.0
argon2-cffi==21.3.0
incremental==21.3.0
packaging==21.3
pip==21.2.4
argon2-cffi-bindings==21.2.0
hyperlink==21.0.0
pyopenssl==21.0.0
automat==20.2.0
black==19.10b0
service-identity==18.1.0
constantly==15.1.0
pyqt5-sip==12.11.1
pyarrow==11.0.0
pillow==9.0.1
ipython==8.2.0
mysql-connector-python==8.1.0
click==8.0.4
tenacity==8.0.1
pycurl==7.44.1
ipywidgets==7.6.5
pytest==7.1.1
ipykernel==6.9.1
notebook==6.4.8
nbconvert==6.4.4
jupyter-console==6.4.0
jupyter-client==6.1.12
pydocstyle==6.1.1
tornado==6.1
pyyaml==6.0
pyqt5==5.15.9
pyqtwebengine==5.15.6
pyqt5-qt5==5.15.2
pyqtwebengine-qt5==5.15.2
isort==5.9.3
psutil==5.8.0
plotly==5.6.0
zope.interface==5.4.0
